In [1]:
import os

In [2]:
photo_list = []
for f in os.listdir("/content/drive/MyDrive/dataset/container"):
    if 'jpg' in f:
        photo_list.append(f)

In [3]:
photo_size = list(map(lambda x: os.path.getsize('/content/drive/MyDrive/dataset/container' + '/' + x), photo_list))

In [4]:
import pandas as pd
fsp = pd.DataFrame({'filename_raw':photo_list, 'size' : photo_size})
print("사진의 갯수:", len(fsp))

사진의 갯수: 317


In [5]:
fsp

,filename_raw,size
0,1.jpg,14395
1,2.jpg,9157
2,3.jpg,88694
3,4.jpg,117834
4,5.jpg,10370
...,...,...
312,313.jpg,12119
313,314.jpg,10515
314,315.jpg,20177
315,316.jpg,17615


In [6]:
import re
com = re.compile(' \d')
fsp['filename'] = list(map(lambda x: com.sub('',x), photo_list))

In [7]:
pvc = pd.DataFrame({'filename':fsp['filename'].value_counts().index, 'fn_counts':fsp['filename'].value_counts().values})
pvc

,filename,fn_counts
0,270.jpg,1
1,130.jpg,1
2,124.jpg,1
3,220.jpg,1
4,310.jpg,1
...,...,...
312,249.jpg,1
313,229.jpg,1
314,97.jpg,1
315,110.jpg,1


In [8]:
psvc = pd.DataFrame({'size':fsp['size'].value_counts().index, 'size_counts':fsp['size'].value_counts().values})
psvc

,size,size_counts
0,7748,2
1,11424,2
2,9581,2
3,6373,2
4,6637,2
...,...,...
307,74584,1
308,13657,1
309,48474,1
310,9566,1


In [9]:
fsp = pd.merge(fsp, pvc, how = 'left', on = 'filename')
fsp = pd.merge(fsp, psvc, how = 'left', on = 'size')

fsp.sample(2)

,filename_raw,size,filename,fn_counts,size_counts
174,175.jpg,9116,175.jpg,1,1
273,274.jpg,9277,274.jpg,1,1


In [10]:
fsp.sample(5)

,filename_raw,size,filename,fn_counts,size_counts
97,98.jpg,105874,98.jpg,1,1
3,4.jpg,117834,4.jpg,1,1
25,26.jpg,73791,26.jpg,1,1
309,310.jpg,11424,310.jpg,1,2
225,226.jpg,7675,226.jpg,1,1


In [11]:
for i in range(len(fsp)):
    if (fsp['fn_counts'][i]) > 1 & (fsp['size_counts'][i] == 1):
        print(i)

130
132
141
170
172
244
264
278
304
309


In [12]:
fsp_nsn = fsp.sort_values(['filename_raw'], ascending = False).drop_duplicates(['filename'], keep='first')

In [13]:
print('남은 사진의 갯수 : {}\n지워진 사진의 갯수 : {}'.format(len(fsp_nsn), len(fsp)-len(fsp_nsn)))

남은 사진의 갯수 : 317
지워진 사진의 갯수 : 0


In [14]:
pvc_nsn = pd.DataFrame({'filename':fsp_nsn['filename'].value_counts().index, 'fn_counts_nsn':fsp_nsn['filename'].value_counts().values})   
psvc_nsn = pd.DataFrame({'size':fsp_nsn['size'].value_counts().index, 'size_counts_nsn':fsp_nsn['size'].value_counts().values})   

fsp_nsn = pd.merge(fsp_nsn, pvc_nsn, how = 'left', on = 'filename')
fsp_nsn = pd.merge(fsp_nsn, psvc_nsn, how = 'left', on = 'size')

In [15]:
fsp_nsn[fsp_nsn['fn_counts_nsn']!=1]

,filename_raw,size,filename,fn_counts,size_counts,fn_counts_nsn,size_counts_nsn


In [16]:
print('사이즈 겹치는 사진의 갯수 :', len(fsp_nsn[fsp_nsn['size_counts_nsn']!=1]))
print('중복 사이즈의 갯수 :', len(psvc_nsn[psvc_nsn['size_counts_nsn']>1]))

사이즈 겹치는 사진의 갯수 : 10
중복 사이즈의 갯수 : 5


In [17]:
!pip install opencv-python
!pip install scikit-image
!pip install imutils

In [18]:
import cv2   # OpenCV
from skimage.measure import compare_ssim

# 삭제될 사진의 리스트
delete = []


for i in range(len(psvc_nsn)):
    
    # 중복된 크기(size)가 있는 경우
    if psvc_nsn['size_counts_nsn'][i] == 2:
        
        # 그 크기에 해당하는 사진을 불러온다. 
        temp = fsp_nsn[fsp_nsn['size']==psvc_nsn['size'][i]].reset_index(drop = True).sort_values(['filename'])
        
        # 사진 읽기
        imageA = cv2.imread('/content/drive/MyDrive/dataset/container/'+temp['filename_raw'][0])
        imageB = cv2.imread('/content/drive/MyDrive/dataset/container/'+temp['filename_raw'][1])
        
        # 이미지를 grayscale로 변환
        grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
        grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
        
        # 이미지의 구조가 같다면 이미지 비교
        if len(grayA)==len(grayB):
            (score, diff) = compare_ssim(grayA, grayB, full=True)
            
            # 차이가 없다면 하나는 delete에 넣어주기
            if score == 1:
                delete.append(temp['filename_raw'][1])
            
            # 구조가 같지만 차이가 존재한다면 직접 확인하기     
            else:
                print('확인해보시오! : ', temp['filename_raw'][0], '/', temp['filename_raw'][1], f'(score : {score})')

확인해보시오! :  245.jpg / 171.jpg (score : 0.23809722621131663)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.


In [19]:
# 중복제거된 것들은 delete 리스트에 넣어주기
delete = delete + list(fsp[~fsp['filename_raw'].isin(fsp_nsn['filename_raw'])]['filename_raw'])

print('삭제할 목록 :', len(delete))

삭제할 목록 : 1


In [20]:
# result : 처음(fsp)데이터에서 - delete를 제외한 것
result = list(fsp[~fsp['filename_raw'].isin(delete)]['filename_raw'])

print('남길 목록 : ', len(result))

남길 목록 :  316


In [22]:
import shutil

for i in result:
    shutil.move('/content/drive/MyDrive/dataset/container/'+i, '/content/drive/MyDrive/dataset/result')
    
for i in delete:
    shutil.move('/content/drive/MyDrive/dataset/container/'+i, '/content/drive/MyDrive/dataset/delete')    